In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df_partitions = pd.read_csv("c3_muse_stress/metadata/partition.csv")
features_path = "c3_muse_stress/feature_segments"
labels_path = "c3_muse_stress/label_segments"

# Load a subject's data
def load_data(id):
    df_bpm = pd.read_csv(f'{features_path}/BPM/{id}.csv').drop(columns=['segment_id'])
    df_ecg = pd.read_csv(f'{features_path}/ECG/{id}.csv').drop(columns=['segment_id'])
    df_resp = pd.read_csv(f'{features_path}/resp/{id}.csv').drop(columns=['segment_id'])
    df_arousal = pd.read_csv(f'{labels_path}/arousal/{id}.csv').drop(columns=['segment_id'])
    df_valence = pd.read_csv(f'{labels_path}/valence/{id}.csv').drop(columns=['segment_id'])
    df_arousal.columns = ['timestamp', 'arousal']
    df_valence.columns = ['timestamp', 'valence']
    return df_bpm, df_ecg, df_resp, df_arousal, df_valence

def merge_dataframes(df_bpm, df_ecg, df_resp, df_arousal, df_valence):
    df = pd.merge_asof(df_bpm, df_ecg, on='timestamp')
    df = pd.merge_asof(df, df_resp, on='timestamp')
    df = pd.merge_asof(df, df_arousal, on='timestamp')
    df = pd.merge_asof(df, df_valence, on='timestamp')
    return df.dropna()

# Split data into train and devel
train_ids = df_partitions[df_partitions['Proposal'] == 'train']['Id']
devel_ids = df_partitions[df_partitions['Proposal'] == 'devel']['Id']

train_data = [merge_dataframes(*load_data(id)) for id in train_ids]
devel_data = [merge_dataframes(*load_data(id)) for id in devel_ids]

train_data = pd.concat(train_data)
devel_data = pd.concat(devel_data)

# Extract features and labels
X_train = train_data[['timestamp', 'BPM', 'ECG', 'resp']].values
y_train = train_data[['timestamp', 'arousal', 'valence']].values
X_devel = devel_data[['timestamp', 'BPM', 'ECG', 'resp']].values
y_devel = devel_data[['timestamp', 'arousal', 'valence']].values

# Remove timestamp column before normalization
X_train = X_train[:, 1:]
y_train = y_train[:, 1:]
X_devel = X_devel[:, 1:]
y_devel = y_devel[:, 1:]

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_devel = scaler.transform(X_devel)

In [32]:
train_ids

0      1
3      4
4      5
7      8
8      9
9     10
10    11
14    16
16    18
18    20
21    23
22    24
25    29
26    30
27    31
28    32
29    33
31    35
32    36
34    38
38    42
39    43
40    44
42    46
43    47
44    48
45    50
47    52
48    53
53    58
54    59
55    60
57    62
58    63
60    65
61    66
63    68
64    69
66    71
67    72
68    73
Name: Id, dtype: int64

In [ ]:
# LSTM

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class PhysiologicalLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob=0.5):
        super(PhysiologicalLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

input_size = 3
hidden_size = 50
num_layers = 2
output_size = 2
dropout_prob = 0.5

model = PhysiologicalLSTMModel(input_size, hidden_size, num_layers, output_size, dropout_prob)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
from torch.utils.data import Dataset, DataLoader

class PhysiologicalSequenceDataset(Dataset):
    def __init__(self, X, y, sequence_length):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.X) - self.sequence_length + 1

    def __getitem__(self, idx):
        return self.X[idx:idx+self.sequence_length], self.y[idx+self.sequence_length-1]

sequence_length = 15  # Example sequence length, adjust as needed

train_dataset = PhysiologicalSequenceDataset(X_train, y_train, sequence_length)
devel_dataset = PhysiologicalSequenceDataset(X_devel, y_devel, sequence_length)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
devel_loader = DataLoader(devel_dataset, batch_size=32, shuffle=False)


In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=20, alpha=1.0):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

def evaluate_model(model, devel_loader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in devel_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
    devel_loss = running_loss / len(devel_loader.dataset)
    print(f'Validation Loss: {devel_loss:.4f}')

train_model(model, train_loader, criterion, optimizer, num_epochs=20)
print()
evaluate_model(model, devel_loader, criterion)


In [ ]:
import numpy as np

model.eval()
predictions = []

with torch.no_grad():
    for inputs, _ in devel_loader:
        outputs = model(inputs)
        predictions.append(outputs.numpy())

predictions = np.vstack(predictions)

ground_truth = np.vstack([labels.numpy() for _, labels in devel_loader])

pred_df = pd.DataFrame(predictions, columns=['Predicted Arousal', 'Predicted Valence'])
gt_df = pd.DataFrame(ground_truth, columns=['True Arousal', 'True Valence'])

timestamps = pd.date_range(start='2022-01-01', periods=len(pred_df), freq='S')

pred_df['timestamp'] = timestamps
gt_df['timestamp'] = timestamps


In [ ]:
# import numpy as np

# window_size = 1
# def smooth_signal(signal, window_size=5):
#     return np.convolve(signal, np.ones(window_size)/window_size, mode='valid')

# smooth_arousal = smooth_signal(pred_df['Predicted Arousal'], window_size=window_size)
# smooth_valence = smooth_signal(pred_df['Predicted Valence'], window_size=window_size)

# # Truncate the timestamps to match the length of the smoothed signals
# pred_df = pred_df.iloc[window_size-1:].copy()
# pred_df['Smoothed Arousal'] = smooth_arousal
# pred_df['Smoothed Valence'] = smooth_valence

# gt_df = gt_df.iloc[:len(pred_df)]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Create traces for arousal
fig.add_trace(go.Scatter(x=gt_df['timestamp'], y=gt_df['True Arousal'], 
                         mode='lines', name='True Arousal'))

fig.add_trace(go.Scatter(x=pred_df['timestamp'], y=pred_df['Predicted Arousal'], 
                         mode='lines', name='Predicted Arousal'))

# Create traces for valence
fig.add_trace(go.Scatter(x=gt_df['timestamp'], y=gt_df['True Valence'], 
                         mode='lines', name='True Valence'))

fig.add_trace(go.Scatter(x=pred_df['timestamp'], y=pred_df['Predicted Valence'], 
                         mode='lines', name='Predicted Valence'))

fig.update_layout(title='True vs Predicted Arousal and Valence',
                  xaxis_title='Timestamp',
                  yaxis_title='Value',
                  legend=dict(x=0.1, y=1.1),
                  legend_orientation="h")

fig.show()


In [ ]:
from torch import nn
import torch
import torch.nn.functional as F
import torch.optim as optim

class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)
        self.fc = nn.Linear(num_channels[-1], output_size)

    def forward(self, x):
        x = x.transpose(1, 2)  # Transpose to (batch_size, channels, seq_length)
        x = self.network(x)
        x = x[:, :, -1]  # Take the last element in the sequence
        x = self.fc(x)
        return x

num_inputs = 3
num_channels = [50] * 2
kernel_size = 2
dropout = 0.2
output_size = 2

model = TemporalConvNet(num_inputs, num_channels, kernel_size=kernel_size, dropout=dropout)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Continue with training and evaluation as previously defined
train_model(model, train_loader, criterion, optimizer, num_epochs=20, alpha=1.0)
evaluate_model(model, devel_loader, criterion, alpha=1.0)
